# Mobile App for Lottery Addiction

Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending from their savings and loans, they start to accumulate debts, and eventually engage in desperate behaviors like theft.

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.


For the first version of the app, they want us to focus on the 6/49 lottery, one of three national lottery games in Canada, and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The dataset is available [here](https://www.kaggle.com/datascienceai/lottery-dataset), and contains data for 3665 drawings, dating from 1982 to 2018.

In [1]:
def factorial(n):
    fact = 1
    while n != 1:
        fact *= n
        n -= 1
    return fact

def combinations(n, k):
    denominator = factorial(k) * factorial(n-k)
    return factorial(n) / denominator

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

In [11]:
def one_ticket_probability(lot_nums):
    c = combinations(49, len(lot_nums))
    p_c = 100 / c
    print('The probability of the combination {} being a winning outcome is {:.7f}%. That means 1 in {:,}.'.format(lot_nums, p_c, int(c)))
    
one_ticket_probability([1,2,3,4,5,6])

The probability of the combination [1, 2, 3, 4, 5, 6] being a winning outcome is 0.0000072%. That means 1 in 13,983,816.


## Initial data exploration

In [5]:
import pandas as pd
lotto_649 = pd.read_csv('649.csv')
print('Dataset contains {0} rows and {1} columns'.format(lotto_649.shape[0], lotto_649.shape[1]))

Dataset contains 3665 rows and 11 columns


In [6]:
lotto_649.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


In [7]:
lotto_649.tail(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


For each drawing, we can find the six numbers drawn in the six columns; "NUMBER DRAWN 1-6"

## Historical combinations

In [14]:
def extract_numbers(row):
    row = row[4:10]
    return set(row)
   
winning_nums = lotto_649.apply(extract_numbers, axis=1)
winning_nums.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

In [19]:
def check_historical_occurence(user_nums, winning_nums):
    user_nums_set = set(user_nums)
    matches = winning_nums == user_nums_set
    print('The combination {} was the winning set {} times.'.format(user_nums, matches.sum()))
#     print('No matter how many times this combination may have occured in the past, the chance of this combination leading to a winning outcome remains the same.')
    one_ticket_probability(user_nums)

In [20]:
check_historical_occurence([14,24,31,35,37,48], winning_nums)

The combination [14, 24, 31, 35, 37, 48] was the winning set 1 times.
The probability of the combination [14, 24, 31, 35, 37, 48] being a winning outcome is 0.0000072%. That means 1 in 13,983,816.


This step compares any possible combination with historical data, informing users about the independence of each lotto draw. There may be a misconception to some users who believe that previous occurences of a combination leads to a higher likelihood of a successful outcome in the future.  

## Multi-ticket probability

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning 


In [37]:
def multi_ticket_probability(n_tickets):
    c = combinations(49, 6)
    p_winning = 100 * n_tickets / c
    print('The proability of winning with {} tickets, is {:.7f}%'.format(int(n_tickets), p_winning))
    print('That is the equivalent of 1 in {:,}'.format(round(c/n_tickets)))
    print('----------------------------------------')

In [38]:
tix = [1, 10, 100, 10000, 1000000, 6991908, 13983816]
for t in tix:
    multi_ticket_probability(t)

The proability of winning with 1 tickets, is 0.0000072%
That is the equivalent of 1 in 13,983,816
----------------------------------------
The proability of winning with 10 tickets, is 0.0000715%
That is the equivalent of 1 in 1,398,382
----------------------------------------
The proability of winning with 100 tickets, is 0.0007151%
That is the equivalent of 1 in 139,838
----------------------------------------
The proability of winning with 10000 tickets, is 0.0715112%
That is the equivalent of 1 in 1,398
----------------------------------------
The proability of winning with 1000000 tickets, is 7.1511238%
That is the equivalent of 1 in 14
----------------------------------------
The proability of winning with 6991908 tickets, is 50.0000000%
That is the equivalent of 1 in 2
----------------------------------------
The proability of winning with 13983816 tickets, is 100.0000000%
That is the equivalent of 1 in 1
----------------------------------------


This shows users that the only real way to improve the statistical probability of a successful outcome is to purchase more tickets. 

## Less winning numbers

In most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

In [39]:
def probability_less_6(matches):
    c_all = combinations(49, 6) 
    c_ticket = combinations(6, matches)           
    c_other = combinations(49-matches, 6-matches)
    winning_outcomes = c_ticket * c_other
    
    p_matches = 100 * winning_outcomes / c_all
    print('The proability of a ticket with {} matching numbers, is {:.7f}%'.format(int(matches), p_matches))
    print('That is the equivalent of 1 in {:,}'.format(round(c_all/winning_outcomes)))
    print('----------------------------------------')

In [40]:
matches = [2, 3, 4, 5]
for m in matches:
    probability_less_6(m)

The proability of a ticket with 2 matching numbers, is 19.1326531%
That is the equivalent of 1 in 5
----------------------------------------
The proability of a ticket with 3 matching numbers, is 2.1710812%
That is the equivalent of 1 in 46
----------------------------------------
The proability of a ticket with 4 matching numbers, is 0.1061942%
That is the equivalent of 1 in 942
----------------------------------------
The proability of a ticket with 5 matching numbers, is 0.0018879%
That is the equivalent of 1 in 52,969
----------------------------------------


## Next steps...

Possible features for a second version of the app include:

- Making the outputs even easier to understand by adding fun analogies (for example, we can find probabilities for strange events and compare with the chances of winning in lottery; for instance, we can output something along the lines "You are 100 times more likely to be the victim of a shark attack than winning the lottery").
- Combining the one_ticket_probability() and check_historical_occurrence() to output information on probability and historical occurrence at the same time.